In [ ]:
import math
import sys
import os

import IPython
import matplotlib.pylab as plt
import numpy as np
from scipy import signal

%load_ext autoreload
%autoreload 2

In [ ]:
from crazyflie_description_py.parameters import TUKEY_ALPHA, N_BUFFER

buffer_length = N_BUFFER # was 2048
fade_length_ratio = 0.2


In [ ]:
# Ramp
fade_length = np.ceil(buffer_length * fade_length_ratio)
print(f"Fade length is {fade_length} samples on a buffer of {buffer_length}")
fade_in = np.arange(0, 1, 1 / (buffer_length * fade_length_ratio))
constant_section = np.ones(int(buffer_length - 2 * fade_length))
ramp_window = np.concatenate([fade_in, constant_section, 1 - fade_in])

In [ ]:
# Tukey
tukey_window = signal.tukey(buffer_length, alpha=TUKEY_ALPHA)
flattop_window = signal.flattop(buffer_length, True)

In [ ]:
plt.plot(ramp_window, label="ramp")
plt.plot(tukey_window, label="tukey")
plt.plot(flattop_window, label="flattop")
plt.legend()

In [ ]:
FS = 32000
f = 1000
noise = 1.0
s = np.sin(2*np.pi*f*np.arange(buffer_length)/FS) + np.random.normal(scale=noise, size=buffer_length)

freq = np.fft.rfftfreq(len(ramp_window), 1/FS)
plt.semilogy(freq, np.abs(np.fft.rfft(s*ramp_window))/buffer_length)
plt.semilogy(freq, np.abs(np.fft.rfft(s*tukey_window))/buffer_length)
plt.semilogy(freq, np.abs(np.fft.rfft(s*flattop_window))/buffer_length)
plt.xlim(f-500, f+500)

In [ ]:
out_dir = "../firmware/audio_shield_firmware/Core/Inc"

save_dict = {
    "tukey_window": tukey_window,
    "flattop_window": flattop_window,
}

for key, arr in save_dict.items():
    fname = f"{out_dir}/{key}.h"
    with open(fname, "w+") as f:
        f.write(f"#ifndef __{key.upper()}_H \n#define __{key.upper()}_H\n\n")

        f.write(f"float32_t tapering_window[{buffer_length}] = " + r"{")
        [f.write(f"{s:.15f}, ",) for s in arr[:-1]]
        [f.write(f"{s:.15f}",) for s in arr[-1:]]
        f.write(r"};" + " \n\n")

        f.write(f"#endif /* __{key.upper()}_H */")
    print('wrote to', fname)